In [2]:
import os,sys
sys.path.append(os.path.abspath('..'))
from jittorKLike.utils import cAxis,silent_preload_jittor
silent_preload_jittor()
import jittor as jt
from jittor import nn
import abc
import numpy as np
from functools import partial,wraps
# import logging

In [3]:
class MetricResultStorage(object):
    def __init__(self,reduction='mean'):
        '''
        args:
          reducetion: default is np.mean
        '''
        super().__init__()
        allowed_reduction={
            'mean':np.mean,
            'sum':np.sum
        }
        self.reset()
        self.reduction=allowed_reduction[str.lower(reduction)]
        
    def reset(self):
        self.results=[]
        self.cnt=0
        
    def push(self,result):
        self.results.append(result)
        self.cnt+=1
        
    @property
    def result(self):
        return self.reduction(self.results)

In [4]:
class Metric(abc.ABC):
    @wraps(MetricResultStorage.__init__, assigned=[],updated=['__annotations__'])
    def __init__(self,*args,**kwargs):
        super().__init__()
        self._results=MetricResultStorage(*args,**kwargs)
    
    @abc.abstractmethod
    def cal_metric(self,*args,**kwargs):
        pass
    
    @property
    def result(self):
        return self._results.result
    
    def reset_state(self):
        self._results.reset()

In [ ]:
class Losses(Metric):
    def cal_metric(self,loss):
        return np.sum(loss)
    
    def update_state(self,loss):
        loss_=self.cal_metric(loss)
        self._results.push(loss_)
        return loss_

In [50]:
class Accuracy(Metric):
    def __init__(self,*args,**kwargs):
        #考虑已经转为one_hot label
        super().__init__(*args,**kwargs)
    
    def _equal_to_int(self,a,b,dtype=jt.int16):
        if a.shape != b.shape:
            raise ValueError("Accuracy::equal: "\
                             "Except args to have the same shape, "\
                            f'but receive {a.shape} and {b.shape}')
        return jt.cast(jt.equal(a,b) ,dtype)
    
    def cal_metric(self,y_true,y_pred):
        gt=jt.argmax(y_true,dim=cAxis)[0]
        pred=jt.argmax(y_pred,dim=cAxis)[0]
        acc=self._equal_to_int(gt,pred, jt.int16)
        return jt.mean(acc)
    
    def update_state(self,y_true,y_pred):
        acc=self.cal_metric(y_true,y_pred)
        self._results.push(acc.numpy())
        return acc

In [27]:
class SparseAccuracy(Accuracy):
    def __init__(self):
        # y非one_hot_label
        super().__init__()
    
    def cal_metric(self,y_true,y_pred):
        gt=y_true
        pred=jt.argmax(y_pred,dim=cAxis)[0]
        acc=self._equal_to_int(gt,pred, jt.int16)
        return jt.mean(acc)

In [22]:
# class A(Metric,MetricResultStorage):
#     def __init__(self):
        
#         print(super())
#         print(super(Metric,self))
#         super().__init__()
    

# A()
# A.mro()

<super: <class 'A'>, <A object>>
<super: <class 'Metric'>, <A object>>
<super: <class 'Metric'>, <A object>>
storage
metric


[__main__.A, __main__.Metric, abc.ABC, __main__.MetricResultStorage, object]